In [1]:
%load_ext autoreload
%autoreload 2
%load_ext tensorboard

In [2]:
from random import random

from SourceCodeTools.models.training_config import get_config, save_config, load_config
from SourceCodeTools.code.data.dataset.Dataset import SourceGraphDataset, filter_dst_by_freq
from SourceCodeTools.models.graph.train.sampling_multitask2 import training_procedure, SamplingMultitaskTrainer
from SourceCodeTools.models.graph.train.objectives.NodeClassificationObjective import NodeClassifierObjective
from SourceCodeTools.models.graph.train.objectives.SubgraphClassifierObjective import SubgraphAbstractObjective, \
    SubgraphClassifierObjective, SubgraphEmbeddingObjective
from SourceCodeTools.models.graph.train.objectives.SubgraphEmbedderObjective import SubgraphEmbeddingObjective, \
    SubgraphMatchingObjective
from SourceCodeTools.models.graph.train.utils import get_name, get_model_base
from SourceCodeTools.models.graph import RGGAN
from SourceCodeTools.tabular.common import compact_property
from SourceCodeTools.code.data.file_utils import unpersist
from SourceCodeTools.models.graph.train.objectives.SCAAClassifierObjetive import SCAAClassifierObjective


import dgl
import torch
import numpy as np
from argparse import Namespace
from torch import nn
from datetime import datetime
from os.path import join
from functools import partial

# Prepare parameters and options

Full list of options that can be added can be found in `SourceCodeTools/models/training_options.py`. They are ment to be used as arguments for cli trainer. Trainer script can be found in `SourceCodeTools/scripts/train.py`.

For the task of subgraph classification the important options are:
- `subgraph_partition` is path to subgraph-based train/val/test sets. Storead as Dataframe with subgraph id and partition mask
- `subgraph_id_column` is a column is `common_edges` file that stores subgraph id.
- For variable misuse task (same will apply to authorship attribution) subgraphs are created for individual functions (files for SCAA). The label is stored in `common_filecontent`.

In [3]:
tokenizer_path = "sentencepiece_bpe.model"

data_path = "./examples/one_to_one/with_ast"
subgraph_partition = join(data_path, "subgraph_partition.json.bz2")
filecontent_path = join(data_path, "common_filecontent.json.bz2")

In [4]:
unpersist(subgraph_partition)

,id,train_mask,val_mask,test_mask
0,2498,True,False,False
1,4644,True,False,False
2,4864,True,False,False
3,270,True,False,False
4,5400,False,False,True
...,...,...,...,...
5533,289,True,False,False
5534,2556,True,False,False
5535,315,True,False,False
5536,1531,True,False,False


In [5]:
unpersist(join(data_path, "common_edges.json.bz2"), nrows=10)

,id,type,source_node_id,target_node_id,file_id,mentioned_in
0,0,subword,213905,516482,2498,NaN
1,1,func,516482,257761,2498,46283.0
2,2,func_rev,257761,516482,2498,46283.0
3,3,value,257761,107985,2498,46283.0
4,4,value_rev,107985,257761,2498,46283.0
...,...,...,...,...,...,...
285,2961294,prev,208030,698551,5397,693983.0
286,2961295,defined_in_module,208030,693983,5397,693983.0
287,2961296,defined_in_module_rev,693983,208030,5397,693983.0
288,2961297,subword,708231,433611,5397,NaN


In [6]:
config = get_config(
    subgraph_id_column="file_id",
    data_path=data_path,
    model_output_dir=data_path,
    subgraph_partition=subgraph_partition,
    tokenizer_path=tokenizer_path,
    objectives="subgraph_clf",
    measure_scores=True,
    dilate_scores=1,
    epochs=100,
)

In [7]:
save_config(config, "var_misuse_tiny.yaml")

In [8]:
config

{'DATASET': {'data_path': './examples/one_to_one/with_ast',
  'train_frac': 0.9,
  'filter_edges': None,
  'min_count_for_objectives': 5,
  'self_loops': False,
  'use_node_types': False,
  'use_edge_types': False,
  'no_global_edges': False,
  'remove_reverse': False,
  'custom_reverse': None,
  'restricted_id_pool': None,
  'random_seed': None,
  'subgraph_id_column': 'file_id',
  'subgraph_partition': './examples/one_to_one/with_ast/subgraph_partition.json.bz2'},
 'TRAINING': {'model_output_dir': './examples/one_to_one/with_ast',
  'pretrained': None,
  'pretraining_phase': 0,
  'sampling_neighbourhood_size': 10,
  'neg_sampling_factor': 3,
  'use_layer_scheduling': False,
  'schedule_layers_every': 10,
  'elem_emb_size': 100,
  'embedding_table_size': 200000,
  'epochs': 100,
  'batch_size': 128,
  'learning_rate': 0.001,
  'objectives': 'subgraph_clf',
  'save_each_epoch': False,
  'save_checkpoints': True,
  'early_stopping': False,
  'early_stopping_tolerance': 20,
  'force_w2v_

# Create Dataset

In [9]:
dataset = SourceGraphDataset(
    **{**config["DATASET"], **config["TOKENIZER"]}
)

/home/alina/workspace/scaa/method-embedding/SourceCodeTools/code/data/dataset/Dataset.py:129: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.nodes = self.nodes.append(new_nodes, ignore_index=True)
/home/alina/workspace/scaa/method-embedding/SourceCodeTools/code/data/dataset/Dataset.py:130: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.edges = self.edges.append(new_edges, ignore_index=True)


# Declare target loading function (labels)

In [10]:
def load_labels():
    filecontent = unpersist(filecontent_path)
    return filecontent[["id", "user"]].rename({"id": "src", "user": "dst"}, axis=1)

In [11]:
load_labels()

,src,dst
0,2498,alex1770
1,4644,sashadotcom
2,4864,superkjelle
3,270,BananaNosh
4,5400,TinyTitans
...,...,...
5533,289,Basti_SUI
5534,2556,anari
5535,315,Bhavyalekha
5536,1531,Poornachander


One or several objectives could be used

In [12]:
class Trainer(SamplingMultitaskTrainer):
    def create_objectives(self, dataset, tokenizer_path):
        self.objectives = nn.ModuleList()
        
        self.objectives.append(
            self._create_subgraph_objective(
                objective_name="SCAAMatching",
                objective_class=SCAAClassifierObjective,
                dataset=dataset,
                tokenizer_path=tokenizer_path,
                labels_fn=load_labels,
            )
        )

In [13]:
# %tensorboard --logdir data
from tensorboard import program

tracking_address = './examples/one_to_one/' # the path of your log file.

tb = program.TensorBoard()
tb.configure(argv=[None, '--logdir', tracking_address])
url = tb.launch()
print(f"Tensorflow listening on {url}")


NOTE: Using experimental fast data loading logic. To disable, pass
    "--load_fast=false" and report issues on GitHub. More details:
    https://github.com/tensorflow/tensorboard/issues/4784



Tensorflow listening on http://localhost:6006/


In [14]:
training_procedure(
    dataset, 
    model_name=RGGAN, 
    model_params=config["MODEL"],
    trainer_params=config["TRAINING"],
    model_base_path=get_model_base(config["TRAINING"], get_name(RGGAN, str(datetime.now()))),
    trainer=Trainer
)

Precompute Target Embeddings:   0%|                                               | 0/44 [00:00<?, ?it/s]

Number of nodes 758247


Epoch 1:   0%|                                                                    | 0/42 [00:00<?, ?it/s]

Epoch: 0, Time: 563 s
{'Accuracy/test/SCAAMatching_': 0.5,
 'Accuracy/train/SCAAMatching_': 0.5,
 'Accuracy/train_avg/SCAAMatching': 0.5,
 'Accuracy/val/SCAAMatching_': 0.5,
 'Loss/test/SCAAMatching_': 0.5975664258003235,
 'Loss/train/SCAAMatching_': 0.6054030060768127,
 'Loss/train_avg/SCAAMatching': 0.5989708843685332,
 'Loss/val/SCAAMatching_': 0.603316605091095,
 'hits@1/test/SCAAMatching_': 0.008771929824561403,
 'hits@1/val/SCAAMatching_': 0.017543859649122806,
 'hits@10/test/SCAAMatching_': 0.010964912280701754,
 'hits@10/val/SCAAMatching_': 0.013157894736842105,
 'hits@3/test/SCAAMatching_': 0.005847953216374269,
 'hits@3/val/SCAAMatching_': 0.005847953216374269,
 'hits@5/test/SCAAMatching_': 0.007017543859649123,
 'hits@5/val/SCAAMatching_': 0.0035087719298245615,
 'map/test/SCAAMatching_': 0.002411249912586906,
 'map/val/SCAAMatching_': 0.0023599585930318577,
 'mr/test/SCAAMatching_': 2445.3684210526317,
 'mr/val/SCAAMatching_': 2444.815789473684,
 'mrr/test/SCAAMatching_': 0

Epoch 2:   0%|                                                                    | 0/42 [00:00<?, ?it/s]

...

Precompute Target Embeddings:   0%|                                               | 0/44 [00:00<?, ?it/s]

Epoch: 99, Time: 556 s
{'Accuracy/test/SCAAMatching_': 0.5,
 'Accuracy/train/SCAAMatching_': 0.5,
 'Accuracy/train_avg/SCAAMatching': 0.5,
 'Accuracy/val/SCAAMatching_': 0.5,
 'Loss/test/SCAAMatching_': 0.6224255561828613,
 'Loss/train/SCAAMatching_': 0.6072116494178772,
 'Loss/train_avg/SCAAMatching': 0.5210327129988443,
 'Loss/val/SCAAMatching_': 0.672966480255127,
 'hits@1/test/SCAAMatching_': 0.008771929824561403,
 'hits@1/val/SCAAMatching_': 0.017543859649122806,
 'hits@10/test/SCAAMatching_': 0.0010964912280701754,
 'hits@10/val/SCAAMatching_': 0.007675438596491228,
 'hits@3/test/SCAAMatching_': 0.0029239766081871343,
 'hits@3/val/SCAAMatching_': 0.005847953216374269,
 'hits@5/test/SCAAMatching_': 0.0017543859649122807,
 'hits@5/val/SCAAMatching_': 0.0035087719298245615,
 'map/test/SCAAMatching_': 0.0029677498545584184,
 'map/val/SCAAMatching_': 0.0018253438475458567,
 'mr/test/SCAAMatching_': 2401.0877192982457,
 'mr/val/SCAAMatching_': 2489.7894736842104,
 'mrr/test/SCAAMatchin

100%|██████████████████████████████████████████████████████████████████████| 1/1 [00:12<00:00, 12.80s/it]

Final eval: hits@1/val/SCAAMatching_final: 0.017543859649122806, hits@3/val/SCAAMatching_final: 0.005847953216374269, hits@5/val/SCAAMatching_final: 0.0035087719298245615, hits@10/val/SCAAMatching_final: 0.007675438596491228, ndcg@1/val/SCAAMatching_final: 0.0009166720128654452, ndcg@3/val/SCAAMatching_final: 0.0009166720128654452, ndcg@5/val/SCAAMatching_final: 0.0009166720128654452, ndcg@10/val/SCAAMatching_final: 0.0010534946417378851, mr/val/SCAAMatching_final: 2489.7894736842104, mrr/val/SCAAMatching_final: 0.0007184399455399677, map/val/SCAAMatching_final: 0.0018253438475458567, scoring_time/val/SCAAMatching_final: 1.6374342441558838, Loss/val/SCAAMatching_final: 0.6164161562919617, Accuracy/val/SCAAMatching_final: 0.5, hits@1/test/SCAAMatching_final: 0.008771929824561403, hits@3/test/SCAAMatching_final: 0.0029239766081871343, hits@5/test/SCAAMatching_final: 0.0017543859649122807, hits@10/test/SCAAMatching_final: 0.0010964912280701754, ndcg@1/test/SCAAMatching_final: 0.0041779190

(<__main__.Trainer at 0x7f76cc919e80>,
 {'hits@1/val/SCAAMatching_final': 0.017543859649122806,
  'hits@3/val/SCAAMatching_final': 0.005847953216374269,
  'hits@5/val/SCAAMatching_final': 0.0035087719298245615,
  'hits@10/val/SCAAMatching_final': 0.007675438596491228,
  'ndcg@1/val/SCAAMatching_final': 0.0009166720128654452,
  'ndcg@3/val/SCAAMatching_final': 0.0009166720128654452,
  'ndcg@5/val/SCAAMatching_final': 0.0009166720128654452,
  'ndcg@10/val/SCAAMatching_final': 0.0010534946417378851,
  'mr/val/SCAAMatching_final': 2489.7894736842104,
  'mrr/val/SCAAMatching_final': 0.0007184399455399677,
  'map/val/SCAAMatching_final': 0.0018253438475458567,
  'scoring_time/val/SCAAMatching_final': 1.6374342441558838,
  'Loss/val/SCAAMatching_final': 0.6164161562919617,
  'Accuracy/val/SCAAMatching_final': 0.5,
  'hits@1/test/SCAAMatching_final': 0.008771929824561403,
  'hits@3/test/SCAAMatching_final': 0.0029239766081871343,
  'hits@5/test/SCAAMatching_final': 0.0017543859649122807,
  'hi